In [38]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm

%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

In [39]:
Member = pd.read_excel('../2progs/Excel_Files/A6_MemberRenamed.xlsx')

In [40]:
MemberRaw=Member.copy()

In [41]:
## Cleaning variable inputs
Member.hhHead = MemberRaw.hhHead.map(dict(Yes=1, No=0))
Member.gender = MemberRaw.gender.map(dict(Female=1, Male=0))
Member.anyEducation = MemberRaw.anyEducation.map(dict(Yes=1, No=0))
Member.speaks = MemberRaw.speaks.map(dict(Yes=1, No=0))
Member.reads = MemberRaw.reads.map(dict(Yes=1, No=0))
Member.writes = MemberRaw.writes.map(dict(Yes=1, No=0))                                    
                                     


In [42]:
## Age Manipulation
ageDict = {"00 to 04": 0,
           "05 to 09": 1,
           "10 to 14": 2,
           "15 to 19": 3,
           "20 to 24": 4,
           "25 to 29": 5,
           "30 to 44": 6,
           "35 to 39": 7,
           "40 to 44": 8,
           "45 to 49": 9,
           "50 to 54": 10,
           "55 to 59": 11,
           "60 or more": 12}

Member.ageRange=Member.ageRange.replace(ageDict)
Member.ageRange=Member.ageRange.fillna(13)


In [43]:
# combine highest education and current grade
Member.highestEducation = Member.highestEducation.fillna(Member.currentGrade)
Member.highestEducation = Member.highestEducation.fillna(0)
# turn into numbers
eduDict = {"Early childhood education":1,
           "Primary": 2,
           "Secondary": 3,
           "Vocational training (more than 1 year, with government certificate)":4,
           "University":5,
           "Other": 6}

Member.highestEducation=Member.highestEducation.replace(eduDict)


In [44]:
# Create DF to test data regarding head of household
HeadMember = Member.query('hhHead==1')


foo = pd.DataFrame({
# Column to count hhsize
    'size': Member.groupby('hhid')['gender'].count(),
# Colomn for houshold head is female 
    'headFemale':HeadMember.groupby('hhid')['gender'].mean(),
# Column for share of house is female
    'shareFemale':Member.groupby('hhid')['gender'].mean(),
# Share by Age stuff
    'shareAgeZeroToFour':Member.query('ageRange==0').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeFiveToNine':Member.query('ageRange==1').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeTenToFourteen':Member.query('ageRange==2').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeFifteenToTwentyfour':Member.query('ageRange==3|ageRange==4').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeTwentyfivetofourtynine':Member.query('ageRange==5|ageRange==6|ageRange==7|ageRange==8|ageRange==9').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeFiftyPlus':Member.query('ageRange==10|ageRange==11|ageRange==12').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareMaleAgeTenToNineteen':Member.query('gender==0&(ageRange==2|ageRange==3)').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareFemaleAgeTenToNineteen':Member.query('gender==1&(ageRange==2|ageRange==3)').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareMaleAgeTwentytoFourtynine':Member.query('gender==0&(ageRange==4|ageRange==5|ageRange==6|ageRange==7|ageRange==8|ageRange==9)').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareFemaleAgeTwentytoFourtynine':Member.query('gender==1&(ageRange==4|ageRange==5|ageRange==6|ageRange==7|ageRange==8|ageRange==9)').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
    'shareAgeNaN':Member.query('ageRange==13').
                        groupby('hhid')['ageRange'].count()/
                        Member.groupby('hhid')['gender'].count(),
# Highest Education Shares
    'shareWithNoEducation': Member.query('highestEducation==0 & ageRange>=4').
                        groupby('hhid')['highestEducation'].count()/
                        Member.query('ageRange>=4').
                            groupby('hhid')['ageRange'].count(),
    'shareWithEarlyChildhood': Member.query('highestEducation==1 & ageRange>=4').
                        groupby('hhid')['highestEducation'].count()/
                        Member.query('ageRange>=4').
                            groupby('hhid')['ageRange'].count(),
    'shareWithPrimary':Member.query('highestEducation==2 & ageRange>=4').
                        groupby('hhid')['highestEducation'].count()/
                        Member.query('ageRange>=4').
                            groupby('hhid')['ageRange'].count(),
    'shareWithSecondary':Member.query('highestEducation==3 & ageRange>=4').
                        groupby('hhid')['highestEducation'].count()/
                        Member.query('ageRange>=4').
                            groupby('hhid')['ageRange'].count(),
    'shareWithVocational':Member.query('highestEducation==4 & ageRange>=4').
                        groupby('hhid')['highestEducation'].count()/
                        Member.query('ageRange>=4').
                            groupby('hhid')['ageRange'].count(),
    'shareWithUniversity':Member.query('highestEducation==5 & ageRange>=4').
                        groupby('hhid')['highestEducation'].count()/
                        Member.query('ageRange>=4').
                            groupby('hhid')['ageRange'].count(),
    'shareWithOther':Member.query('highestEducation==6 & ageRange>=4').
                        groupby('hhid')['highestEducation'].count()/
                        Member.query('ageRange>=4').
                            groupby('hhid')['ageRange'].count(),
# Share 5-10 that speaks
    'shareSpeaks': Member.query('speaks==1 & ageRange==1').
                        groupby('hhid')['speaks'].count()/
                        Member.query('ageRange==1').
                            groupby('hhid')['ageRange'].count(),
# Reads
    'shareReads':Member.query('reads==1 & ageRange>=4').
                        groupby('hhid')['highestEducation'].count()/
                        Member.query('ageRange>=4').
                            groupby('hhid')['ageRange'].count(),
    'shareWrites':Member.query('writes==1 & ageRange>=4').
                        groupby('hhid')['highestEducation'].count()/
                        Member.query('ageRange>=4').
                            groupby('hhid')['ageRange'].count(),
    }).fillna(0)

# change it to NA if there is no person for the last ones for share speaks

foo.shareSpeaks.min()


0.0

In [45]:
foo.query('shareAgeNaN!=0.0')

,size,headFemale,shareFemale,shareAgeZeroToFour,shareAgeFiveToNine,shareAgeTenToFourteen,shareAgeFifteenToTwentyfour,shareAgeTwentyfivetofourtynine,shareAgeFiftyPlus,shareMaleAgeTenToNineteen,...,shareWithNoEducation,shareWithEarlyChildhood,shareWithPrimary,shareWithSecondary,shareWithVocational,shareWithUniversity,shareWithOther,shareSpeaks,shareReads,shareWrites
hhid,,,,,,,,,,,,,,,,,,,,,
72,8,1,0.375000,0.375000,0.250000,0.000000,0.000000,0.250000,0.0,0.000000,...,0.666667,0.0,0.000000,0.333333,0.0,0.0,0.000000,1.0,0.333333,0.333333
184,5,1,0.600000,0.200000,0.200000,0.000000,0.000000,0.400000,0.0,0.000000,...,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.000000,0.000000
211,8,0,0.625000,0.250000,0.250000,0.125000,0.000000,0.125000,0.0,0.000000,...,0.333333,0.0,0.000000,0.333333,0.0,0.0,0.333333,0.0,0.666667,0.666667
238,12,0,0.416667,0.083333,0.250000,0.166667,0.250000,0.166667,0.0,0.083333,...,0.250000,0.0,0.250000,0.250000,0.0,0.0,0.250000,1.0,0.500000,0.500000
251,9,1,0.333333,0.000000,0.333333,0.333333,0.111111,0.111111,0.0,0.333333,...,0.500000,0.0,0.000000,0.500000,0.0,0.0,0.000000,1.0,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5591,7,0,0.714286,0.142857,0.142857,0.428571,0.000000,0.142857,0.0,0.142857,...,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.000000,1.0,1.000000,1.000000
5643,9,0,0.777778,0.333333,0.111111,0.222222,0.000000,0.222222,0.0,0.000000,...,0.666667,0.0,0.333333,0.000000,0.0,0.0,0.000000,1.0,0.666667,0.666667
5666,4,1,0.750000,0.250000,0.250000,0.000000,0.000000,0.250000,0.0,0.000000,...,0.000000,0.0,0.500000,0.500000,0.0,0.0,0.000000,1.0,0.500000,0.500000


In [46]:
table1 = foo[['headFemale', 'shareFemale','shareAgeZeroToFour','shareAgeFiveToNine', 'shareAgeTenToFourteen', 
    'shareAgeFifteenToTwentyfour', 'shareAgeTwentyfivetofourtynine', 'shareAgeFiftyPlus','shareMaleAgeTenToNineteen',
    'shareFemaleAgeTenToNineteen', 'shareMaleAgeTwentytoFourtynine', 'shareFemaleAgeTwentytoFourtynine',
    'shareWithNoEducation', 'shareWithEarlyChildhood', 'shareWithPrimary', 'shareWithSecondary', 'shareWithVocational',
    'shareWithUniversity', 'shareWithOther', 'shareSpeaks', 'shareReads', 'shareWrites']].describe().T[['mean', 'std', 'count']].set_axis(['Head Female', 'Share Female', '% Members Aged 0-4', '% Members Aged 5-9',
                                                                                                                                          '% Members Aged 10-14', '% Members Aged 15-24', '% Members Aged 25-49',
                                                                                                                                          '% Members Over Age 50', '% Male Members Aged 10-19', '% Female Members Aged 10-19',
                                                                                                                                          '% Male Members Aged 20-49', '% Female Members Aged 20-49', '% Members with No Education',
                                                                                                                                  '% Members with Early Childhood Education', '% Members with Primary Education',
                                                                                                                                       '% Members with Secondary Education',  '% Members with Vocational Education',
                                                                                                                                           '% Members with University Education',  '% Members with Other Education',
                                                                                                                                          '% Members who Speak', '% Members who Read', '% Members who Write'], axis = 'index').set_axis(['Mean', 'Std', 'N'], axis='columns')                

In [47]:
table1.round(2)

,Mean,Std,N
Head Female,0.66,0.48,1100.0
Share Female,0.49,0.26,1100.0
% Members Aged 0-4,0.19,0.17,1100.0
% Members Aged 5-9,0.19,0.17,1100.0
% Members Aged 10-14,0.15,0.16,1100.0
% Members Aged 15-24,0.22,0.25,1100.0
% Members Aged 25-49,0.22,0.23,1100.0
% Members Over Age 50,0.01,0.07,1100.0
% Male Members Aged 10-19,0.16,0.20,1100.0
% Female Members Aged 10-19,0.12,0.15,1100.0


In [48]:
foo.to_excel('../2progs/Excel_Files/B1_MemberCleaning.xlsx',sheet_name='Sheet1')